#TopEditor - traitements NER et enrichissements des textes TEI

Le dossier du projet se trouve ici :
```
/content/drive/MyDrive/Colab Notebooks/TopEditor/
```
Description des sous-dossiers associés aux traitements:
* **NERmodels** : les modèles NER produits de reperage d'entités nommées (et autres)

* **data/TEI/Final/input**: fichiers TEI stylé via Word customisé Metopes et converti en schéma métopes (via XMLMind)

* **data/TEI/Final/clean**: fichiers TEI du dossier input legerement "nettoyés" manuellement du bruit produit par Word après consignes Métopes

* **data/TEI/Final/tagged**: fichiers TEI avec balisage automatisé via un modèle NER custom

* **data/TEI/Final/reference_pers**: fichiers tableurs avec les identifiants personnes à désambiguiser manuellement

* **data/TEI/Final/reference_place**: fichiers tableurs avec les identifiants place (sens restreint) à désambiguiser manuellement

* **data/TEI/Final/tagged/updated_person**: fichiers dans "tagged" avec les identifiants (attribut ref) pour les personnes

* **data/TEI/Final/tagged/updated_person_upated_place**: fichiers dans "tagged" avec les identifiants (attribut ref) pour les personnes et pour les lieux

* **data/TEI/Final/tagged_corrected_Sara**: fichiers TEI dans updated_person_upated_place avec la correction manuelle par Sara via Oxygène des balises fautives

* **data/TEI/Final/tagged_corrected_Sara/TagAugmented** fichiers dans tagged_corrected_Sara avec un balisage automatisé par reglès des dates déjà reperées par le modèle NER et pas du tout repérées




## Personnes (tagged -> reference_pers, updated_person)

Gestion du réferentiel des personnes citées

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Création d'un tableur de personnes et leurs identifiants à partir des noms de personnes repérés par le NER dans les fichiers (TEI) et réinjection de ces identifiants dans les fichiers TEI/XML

In [ ]:
import os
from lxml import etree
import pandas as pd
import re

def create_and_inject_person_id(paragraph, namespace):
    """Crée un identifiant prenom_nom et l'injecte dans l'attribut 'ref' de chaque balise persName."""
    for persname in paragraph.findall('.//tei:persName', namespaces=namespace):
        next_sibling = persname.getnext()
        alias = ""
        persname_text = get_full_text(persname)
        if next_sibling is not None and next_sibling.tag == f"{{{namespace['tei']}}}name":
          alias = get_full_text(next_sibling)
        if alias:
          alias = alias.replace(" ", "_")
          person_id = f"{persname_text}_{alias}".replace(" ", "_")
        else:
          person_id = f"{persname_text}".replace(" ", "_")
        persname.set('ref', "#"+person_id)

def get_full_text(element):
    """Récupère tout le texte (y compris sous-éléments) dans une balise."""
    parts = [element.text] if element.text else []
    for sub_element in element:
        parts.append(get_full_text(sub_element))
        if sub_element.tail:
            parts.append(sub_element.tail)
    return ''.join(parts).strip()

def parse_xml_file(file_path, namespace):
    """Charge un fichier XML et retourne l'arborescence et le body."""
    tree = etree.parse(file_path)
    root = tree.getroot()
    body = root.find('.//tei:body', namespaces=namespace)
    return body

def extract_code_from_paragraph(paragraph_text):
    """Extrait et nettoie le code au début du paragraphe."""
    match = re.match(r'^(?:CH_[A-Z]{2}_\d+([a-zA-Z\+\(\)\.]|\s\(.+\))*\s?)+', paragraph_text)
    return match.group(0).strip() if match else None

def extract_dates_from_paragraph(paragraph, namespace):
    """Extrait toutes les dates d'un paragraphe."""
    dates = paragraph.findall('.//tei:date', namespaces=namespace)
    return [get_full_text(date) for date in dates]

def extract_street_and_district_from_paragraph(paragraph, namespace):
    """Extrait les noms de rues et de quartiers d'un paragraphe."""
    streets = paragraph.findall('.//tei:rs[@type="street_name"]', namespaces=namespace)
    districts = paragraph.findall('.//tei:rs[@type="area_name"]', namespaces=namespace)
    street_names = [get_full_text(street) for street in streets]
    district_names = [get_full_text(district) for district in districts]
    return street_names, district_names

def extract_paragraph_data(paragraph, namespace):
    """Extrait les données d'un paragraphe donné."""
    paragraph_text = get_full_text(paragraph)
    paragraph_id = paragraph.get('{http://www.w3.org/XML/1998/namespace}id')
    dates = extract_dates_from_paragraph(paragraph, namespace)
    street_names, district_names = extract_street_and_district_from_paragraph(paragraph, namespace)

    data = []
    for persname in paragraph.findall('.//tei:persName', namespaces=namespace):
        persname_text = get_full_text(persname)
        persID = persname_text.replace(" ", "_")

        # Extraction du code CH_BO_00 du début du paragraphe
        code_match = extract_code_from_paragraph(paragraph_text)

        profession = None
        alias = None
        family_role = None
        role_name = None
        placeName = None

        # Trouver une balise après persName
        next_sibling = persname.getnext()
        if next_sibling is not None:
            # cas <persName> + <rs type="profession">
            # ex : <persName ref="x">Alfonso García</persName>, <rs type="profession">tejedor</rs>,
            if next_sibling.tag == f"{{{namespace['tei']}}}rs" and next_sibling.get('type') == 'profession':
                profession = get_full_text(next_sibling)

            # cas <persName> + <name>
            # ex : <persName ref="x">Pedro Gómez</persName><name>Gudiel</name>
            elif next_sibling.tag == f"{{{namespace['tei']}}}name":
                alias = get_full_text(next_sibling)
                next_next_sibling = next_sibling.getnext()
                persID = f"{persname_text}_{alias}".replace(" ", "_")
                #cas <persName> + <name> + <rs type="profession">
                # ex : <persName ref="x">Juan Fernández</persName><name>de Mora</name>, <rs type="profession">sofiel</rs>
                if next_next_sibling is not None and next_next_sibling.tag == f"{{{namespace['tei']}}}rs" and next_next_sibling.get('type') == 'profession':
                    profession = get_full_text(next_next_sibling)

            # cas <persName> + <rs type="family_role">
            # ex : <persName ref="x">Francisca Fernández</persName>, sa <rs type="family_role">femme</rs>
            elif next_sibling.tag == f"{{{namespace['tei']}}}rs" and next_sibling.get('type') == 'family_role':
                family_role = get_full_text(next_sibling)

            # cas <persName> + <roleName>
            # ex : <persName ref="x">Alfonso Fernández</persName>, lequel était <roleName>chanoine obrero</roleName>
            elif next_sibling.tag == f"{{{namespace['tei']}}}roleName":
                role_name = get_full_text(next_sibling)

                # cas <persName> + <roleName> + <placeName>
                # <persName ref="x">don Pedro Gómez</persName>, <roleName>archidiacre</roleName> de <placeName>Tolède</placeName>
                next_next_sibling = next_sibling.getnext()
                if next_next_sibling is not None and next_next_sibling.tag == f"{{{namespace['tei']}}}placeName":
                    placeName = get_full_text(next_next_sibling)

        previous_sibling = persname.getprevious()
        if previous_sibling is not None:
          # cas <roleName> + <persName>
          # ex : <roleName>Maestre</roleName><persName ref="x">Jufré</persName>
          if previous_sibling.tag == f"{{{namespace['tei']}}}roleName":
            role_name = get_full_text(previous_sibling)

          # cas <rs type="family_role"> + <persName>
          # ex : <rs type="family_role">femme</rs>, <persName ref="x">Gracia Gutiérrez</persName>
          elif previous_sibling.tag == f"{{{namespace['tei']}}}rs" and previous_sibling.get('type') == 'family_role':
            family_role = get_full_text(previous_sibling)

        # cas general (implicite): <persName>
        # ex : <persName ref="x">don Pedro Gómez</persName>

        data.append({
            'nombre': persname_text,
            'identificador': persID,
            'identificador_desambiguar': persID,
            'alias': alias,
            'oficio': profession,
            'rol_familiar': family_role,
            'rol': role_name,
            'mencion_lugar': placeName,
            'codigo_casa': code_match,
            'fechas': '; '.join(dates),
            'calles': '; '.join(street_names),
            'collaciones': '; '.join(district_names),
            'id_paragrafo': paragraph_id,
            'paragrafo': paragraph_text
        })
    return data

def process_file(file_path, namespace):
    """Traite un fichier XML pour extraire les données et mettre à jour les balises persName."""
    tree = etree.parse(file_path)
    root = tree.getroot()
    body = root.find('.//tei:body', namespaces=namespace)
    if body is None:
        return []

    data = []
    paragraphs = body.findall('.//tei:p', namespaces=namespace)
    for paragraph in paragraphs:
        data.extend(extract_paragraph_data(paragraph, namespace))
        create_and_inject_person_id(paragraph, namespace)

    # Sauvegarder le fichier XML mis à jour
    updated_file_path = file_path.replace('.xml', '_updated_person.xml')
    tree.write(updated_file_path, encoding='utf-8', xml_declaration=True)

    return data

def save_to_excel(data, output_file):
    """Enregistre les données dans un fichier Excel."""
    df = pd.DataFrame(data)
    df.to_excel(output_file, index=False, sheet_name='Occurrences')

def extract_persname_context(xml_folder, out_tableur):
    """Parcourt les fichiers XML dans un dossier et extrait les données pertinentes."""
    namespace = {'tei': 'http://www.tei-c.org/ns/1.0'}

    for file_name in os.listdir(xml_folder):
        if file_name.endswith('_clean.xml'):
            file_path = os.path.join(xml_folder, file_name)
            print(f"Traitement du fichier : {file_path}")

            try:
                data = process_file(file_path, namespace)

                if data:
                    output_file = os.path.join(out_tableur, f"{os.path.splitext(file_name)[0]}_persName_context.xlsx")
                    save_to_excel(data, output_file)
                    print(f"Extraction terminée pour {file_name}. Résultats sauvegardés dans {output_file}")
                else:
                    print(f"Aucune donnée extraite pour {file_name}.")

            except etree.XMLSyntaxError as e:
                print(f"Erreur lors de l'analyse du fichier {file_name}: {e}")

# Paramètres du script
xml_folder = '/content/drive/MyDrive/Colab Notebooks/TopEditor/data/TEI/Final/tagged'
out_tableur = '/content/drive/MyDrive/Colab Notebooks/TopEditor/data/TEI/Final/reference_pers'

extract_persname_context(xml_folder, out_tableur)

Traitement du fichier : /content/drive/MyDrive/Colab Notebooks/TopEditor/data/TEI/Final/tagged/semtags_2combiné_T2 clj. San Pedro_clean.xml
Extraction terminée pour semtags_2combiné_T2 clj. San Pedro_clean.xml. Résultats sauvegardés dans /content/drive/MyDrive/Colab Notebooks/TopEditor/data/TEI/Final/reference_pers/semtags_2combiné_T2 clj. San Pedro_clean_persName_context.xlsx


Ver notebook de "control"

### Après correction manuelle:

#### TODO: Supprimer les faux positifs (pers) signalés par Jean (via tableurs) des fichiers XML via l'identifiant (attribut ref)

In [ ]:
dossier = "/content/drive/MyDrive/Colab Notebooks/TopEditor/data/TEI/Final/reference_pers/Corrected_Jean/"

suffix_files = "_ErrbaliseEN_correctedJean.csv"

ICI

# Supprimer dans des fichiers TEI en entrée, le balisage (et non le contenu) des balises listés dans un CSV d'un segment de texte donné (col 1)
# dans un paragraphe identifié pX (col 13) et  une valeur donnée dans l'attribut ref (col2)

#### TODO Rebaliser les mentions des personnes avec des balises contenantes

Selon ces reglès:

```
Cas recensés traitables et traités (dans l'ordre du code):
<persName ref="x">Alfonso García</persName>, <rs type="profession">tejedor</rs>,
<persName ref="x">Pedro Gómez</persName><name>Gudiel</name>
<persName ref="x">Juan Fernández</persName><name>de Mora</name>, <rs type="profession">sofiel</rs>
<persName ref="x">Francisca Fernández</persName>, sa <rs type="family_role">femme</rs>
<persName ref="x">Alfonso Fernández</persName>, lequel était <roleName>chanoine obrero</roleName>
<roleName>Maestre</roleName><persName ref="x">Jufré</persName>
<persName ref="x">don Pedro Gómez</persName>
<rs type="family_role">femme</rs>, <persName ref="x">Gracia Gutiérrez</persName>
<persName ref="x">don Pedro Gómez</persName>, <roleName>archidiacre</roleName> de <placeName>Tolède</placeName>


Cas pas traités : pas de persName autour
<roleName>archidiacre</roleName> de <placeName>Tolède</placeName>  
<roleName>chanoine obrero</roleName> de l'<orgName>Église</orgName>
<roleName>chanoine obrero</roleName> de l'<orgName>Église</orgName> de <placeName>Tolède</placeName>
<rs type="family_role">sœur</rs> de l’<roleName>archiprêtre</roleName> d’<placeName>Ocaña</placeName>
TODO: entourer ces mentions en persName ?
```

#### TODO Génération de l'index TEI pour les personnes à partir des fichiers TEI et les tableurs corrigés

#### TODO Croissement du tableur de personnes citées et la liste des personnes de Molenat

Créer les fichiers TEI index personnes et lieux (comme topurbi) pour les fichiers 3,4,5 en cours

In [ ]:
pers_molenat_tableur = "/content/drive/MyDrive/Colab Notebooks/TopEditor/data/Textenrichment/Anthroponymes_Molenat.csv"

#### TODO Croissement du tableur de personnes citées et les autorités intégrées dans IDREF

Faire à la fin à partir des listes nettoyées de toutes les personnes ?

In [ ]:
pers_juives_idref = "/content/drive/MyDrive/Colab Notebooks/TopEditor/data/Textenrichment/enrichement_TOPEDITOR_Fourniture_IdRef_merged_12122024_personnes_juives_cleanman.csv"